In [19]:
import torch
import pandas as pd
from torchtext.data.metrics import bleu_score
from tqdm import tqdm

predictions = pd.read_csv('predictions.csv')

In [2]:
def get_bleu_score(predictions):
    scores = []
    ratio = 0.5
    data = predictions[predictions['ratio'] == ratio]
    for vid in tqdm(predictions['vid'].unique()):
        best_score = 0
        outputs = data[data['vid'] == vid]['output'].astype(str).apply(lambda x: x.split())
        comments = data[data['vid'] == vid]['comment'].astype(str).apply(lambda x: x.split())
        for i, output in outputs.iteritems():
            score = bleu_score([output], [list(comments)])
            if score >= best_score:
                best_output = output
                best_score =  score
        scores.append(best_score)
    print(ratio, sum(scores)/len(scores))

In [39]:
def get_best_k_output(predictions, k):
    scores = []
    ratio = 0.5
    model = predictions['model'].iloc[1]
    data = predictions[predictions['ratio'] == ratio]
    for vid in tqdm(predictions['vid'].unique()):
        best_score = 0
        best_outputs = []
        outputs = data[data['vid'] == vid]['output'].astype(str).apply(lambda x: x.split())
        comments = data[data['vid'] == vid]['comment'].astype(str).apply(lambda x: x.split())
        for i, output in outputs.iteritems():
            score = bleu_score([output], [list(comments)])
            if score >= best_score:
                best_output = output
                best_score =  score
                output = ' '.join(output)
                best_outputs.append((output, score))
        with open('best_comment.csv', 'a')as f:
            for output, score in best_outputs[-k:]:
                f.write(','.join([model, vid, output, str(score)]))
                f.write('\n')
                
                

In [40]:
predictions = pd.read_csv('predictions-10.csv')
get_best_k_output(predictions, 10)

100%|██████████| 591/591 [14:49<00:00,  1.50s/it]


In [23]:
predictions = pd.read_csv('predictions-10.csv')
get_bleu_score(predictions)

100%|██████████| 591/591 [14:51<00:00,  1.51s/it]

0.5 0.1914617990890838


In [120]:
se2seq_predictions = pd.read_csv('Models/Seq2seq-predictions.csv')
get_bleu_score(se2seq_predictions)

100%|██████████| 591/591 [23:15<00:00,  2.36s/it]

0.5 0.18952642898187375
